In [ ]:
#Import dependencies
import pandas as pd
import requests
from pprint import pprint
from api_keys import api_key

In [ ]:
#Save API url
url = "https://api-nba-v1.p.rapidapi.com/players/"

#Query for players in the USA 
querystring = {"country":"USA"}

#input header values
headers = {
	
	#input api key from api key file
	"X-RapidAPI-Key": api_key,
	"X-RapidAPI-Host": "api-nba-v1.p.rapidapi.com"
}
#Stores response as a json file in variable response
response = requests.get(url, headers=headers, params=querystring).json()

#pretty print response
pprint(response)

In [ ]:
#saves response list as players 
players_data = response['response']

#create an empty list to store each player dictionaries in 
players = []

#loop through the json list
for player in players_data:

    #add each player dictionary to players list 
    players.append(player)

#Displays first player entry
players[0]

In [ ]:
#Create a dataframe from the players array
players_df = pd.DataFrame(players)

#Display first 5 rows of the df
players_df.head()

In [ ]:
#Check column types
players_df.dtypes

In [ ]:
#change column types to strings
players_df = players_df.convert_dtypes()

#recheck column types
players_df.dtypes


In [ ]:
#create a function to identify columns that are object type
def is_array_of_dicts(column_name):
    #create an empty list
    lib_column_names = []

    #loop through columns in the dataframe
    for column_name in players_df.columns:
        
        #checks if the column type is equal to object
        if players_df[column_name].dtype == 'object':

            #adds the column name to the list
            lib_column_names.append(column_name)
    #return the list of object columns        
    return lib_column_names

#create a function to split the dictionaries in the object columns
def split_and_concat_columns(df, columns_to_split):

    #loop through the given columns
    for name in columns_to_split:

        #splits the library into two columns
        split_columns = df[name].apply(pd.Series)
        
        #organizes new columns to match
        df = pd.concat([df, split_columns], axis=1)
        
        #drops the original column
        df = df.drop(name,axis=1)
    
    #returns the new dataframe
    return df

#run the functions to split the columns with dictionaries as values
players_df = split_and_concat_columns(players_df,is_array_of_dicts(players_df.columns))

#display sample from dataframe
players_df.head()


In [ ]:
#drop new unnecessary columns
players_df = players_df.drop(['meters', 'kilograms', 'vegas', 'africa','sacramento'],axis =1)

#recheck column types
print(players_df.dtypes)

#display new df
players_df.head()

In [ ]:
#change the date column to datetime
players_df['date'] = pd.to_datetime(players_df['date'])

#convert other columns from obects to string if possible
players_df = players_df.convert_dtypes()

#recheck datatypes
print(players_df.dtypes)

#visualize dataframe
players_df.head()

In [ ]:
#split the 'standard' column
players_df = split_and_concat_columns(players_df,is_array_of_dicts(players_df['standard']))

#display dataframe
players_df.head()

In [ ]:
#drop 0 column
players_df = players_df.drop([0],axis =1)

#display dataframe
players_df.head()

In [ ]:
#drop rows with null, none, or <NA> values
players_df = players_df.dropna()

#reset the dataframe index
players_df.reset_index(inplace=True)

#recheck column types
print(players_df.dtypes)

#display dataframe
players_df.head()

In [ ]:
#Change height, weight, and jersy values to integers
players_df['feets'] = players_df['feets'].astype(int)
players_df['inches'] = players_df['inches'].astype(int)
players_df['pounds'] = players_df['pounds'].astype(int)
players_df['jersey'] = players_df['jersey'].astype(int)

#recheck datatypes
print(players_df.dtypes)
#display sample of dataframe
players_df.head()

In [ ]:
#multiply feets values by 12 and add to the inches column to get full height in inches
players_df['height(in)'] = (players_df['feets'] * 12) + players_df['inches']

#drop feets and inches columns
players_df = players_df.drop(['feets','inches'],axis=1)

#display sample of df
players_df.head()